# import

In [35]:
import os
import random

import timm
import wandb
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer, LightningDataModule
from pytorch_lightning.loggers import WandbLogger
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score, f1_score

# Configs

In [54]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
data_path = '../data/'

# model config
model_list = {
    1 : 'resnet34',
    2 : 'resnet50',
    3 : 'resnet101',
    4 : 'resnet152',
    6 : 'vit_small_patch16_224',
    5 : 'vit_base_patch16_224',
    7 : 'convnext_base.fb_in22k_ft_in1k',
    8 : 'vit_large_patch16_224',
    9 : 'convnextv2_huge.fcmae_ft_in1k'	,
}

model_family = {"resnet" : [model_list[1],
                            model_list[2],
                            model_list[3],
                            model_list[4],
                            model_list[7],
                            model_list[9],],
                "vit" : [model_list[6],
                         model_list[5],
                         model_list[8]]
                            }

num_classes=17

# training config

CFS={"MODEL" : model_list[9],
    "IMG_SIZE" : 224,
     "LR" : 1e-4,
    'EPOCHS' : 50,
    'BATCH_SIZE' : 32,
    "NUM_WORKERS" : 16,}

wandb_logger = WandbLogger(
    project="pl-migration",
    name=f"{CFS['MODEL']},{CFS['BATCH_SIZE']},{CFS['EPOCHS']},{CFS['LR']}",
    config=CFS,
)


# Transform

In [ ]:
# 스코어가 하락한 augementation 모음

# A.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, rotate=(-15, 15), scale=(0.9, 1.1), shear=(-10, 10), p=0.5),
# A.Perspective(scale=(0.05, 0.1), p=0.5),
# A.SquareSymmetry(p=0.2),
#        A.Transpose(p=0.5), 




In [55]:
if CFS['MODEL'] in model_family['resnet']:
    norm_mean = [0.485, 0.456, 0.406]
    norm_std = [0.229, 0.224, 0.225]
else:
    norm_mean = [0.5, 0.5, 0.5]
    norm_std = [0.5, 0.5, 0.5]
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 1. 기하학적 변환 (Geometric Transformations)
    A.OneOf([
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.15, rotate_limit=15, p=0.5),
        A.OpticalDistortion(distort_limit=0.2, p=0.5),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5)
    ], p=1.0),
    
    # 2. 공간적 변형 (Spatial Transformations)
    A.OneOf([
        A.RandomCrop(height=int(CFS["IMG_SIZE"]*0.9), width=int(CFS["IMG_SIZE"]*0.9), p=0.7),
        A.RandomResizedCrop(size=(CFS["IMG_SIZE"], CFS["IMG_SIZE"]), scale=(0.8, 1.0), p=0.3),
        A.Transpose(p=0.2), 
        A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.2),
    ], p=1.0),
    
    # 3. 색상 변환 (Color Transformations)
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
        A.RandomGamma(gamma_limit=(80, 120), p=0.3),
        A.CLAHE(clip_limit=4.0, p=0.2)
    ], p=1.0),
    
    # 4. 노이즈 및 블러 (Noise & Blur)
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), mean=0.0, per_channel=True, p=0.4),
        A.GaussianBlur(blur_limit=(3, 7), p=0.3),
        A.MotionBlur(blur_limit=7, p=0.3)
    ], p=1.0),
    
    # 5. 고급 증강 기법 (Advanced Augmentations)
    A.OneOf([
        A.CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.5),
        A.RandomSunFlare(src_radius=100, p=0.1),
        A.RandomShadow(num_shadows_lower=1, num_shadows_upper=3, p=0.2)
    ], p=1.0),
    
    # 6. 최종 전처리
    A.Resize(CFS["IMG_SIZE"], CFS['IMG_SIZE']),
    A.Normalize(mean=norm_mean, std=norm_std),
    ToTensorV2()
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(CFS["IMG_SIZE"], CFS['IMG_SIZE']),
    A.Normalize(mean=norm_mean, std=norm_std),
    ToTensorV2(),
])

# WandB에 로깅
wandb_logger.experiment.config["train_transform"] = str(trn_transform)
wandb_logger.experiment.config["test_transform"] = str(tst_transform)


# print(transform_str)
# WandB에 파라미터 로깅
# wandb_logger.experiment.config["train_transform"] = get_transform_params(trn_transform)
# wandb_logger.experiment.config["test_transform"] = get_transform_params(tst_transform)


/data/ephemeral/home/.pyenv/versions/3.12.10/envs/py12/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_275750/1374115616.py:23: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.2),
/tmp/ipykernel_275750/1374115616.py:35: UserWarning: Argument(s) 'var_limit, mean' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), mean=0.0, per_channel=True, p=0.4),
/tmp/ipykernel_275750/1374115616.py:42: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=16, max_width=16, fill_value=0, p=0.5),
/tmp/ipykernel_275750/1374115616.py:44: UserWarning: Argument(s) 'num_shadows_lower, num_shadows_upper' ar

# Dataset

In [56]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# Datamodule

In [57]:
class DataModule(LightningDataModule):
    def __init__(self, data_path, train_transform, test_transform, batch_size, num_workers):
        super().__init__()
        self.data_path = data_path
        self.train_transform = train_transform
        self.test_transform = test_transform
        self.batch_size = batch_size
        self.num_workers = num_workers

    def setup(self, stage=None):
        if stage == "fit" or stage is None:
            self.train_dataset = ImageDataset(
                csv=os.path.join(self.data_path, "train.csv"),
                path=os.path.join(self.data_path, "train"),
                transform=self.train_transform
            )
            
        if stage == "test" or stage == "predict" or stage is None:
            self.test_dataset = ImageDataset(
                csv=os.path.join(self.data_path, "sample_submission.csv"),
                path=os.path.join(self.data_path, "test"),
                transform=self.test_transform
            )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            pin_memory=True,
            drop_last=False
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True
        )
    
    def predict_dataloader(self):
        return self.test_dataloader()
    
datamodule = DataModule(data_path='../data/',
    train_transform=trn_transform,
    test_transform=tst_transform,
    batch_size=CFS['BATCH_SIZE'],
    num_workers=CFS['NUM_WORKERS']
)

# Model

In [58]:
class LightningModel(LightningModule):
    def __init__(self, model_name, num_classes, lr):  
        super().__init__()
        self.model = timm.create_model(
            model_name,
            pretrained=True,
            num_classes=num_classes 
        )
        self.lr = lr
        self.num_classes = num_classes
        self.train_preds = []      # 예측값 저장
        self.train_targets = []    # 타겟 저장
        self.train_losses = []     # 배치별 손실 저장

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1)
        
        # 배치 단위 로깅
        self.log('train_loss_step', loss, prog_bar=True)
        
        # 에포크 종료 시 메트릭 계산을 위한 데이터 수집
        self.train_preds.append(pred.detach().cpu())
        self.train_targets.append(y.detach().cpu())
        self.train_losses.append(loss.detach().cpu())  # 손실 추가 저장
        
        return loss

    def on_train_epoch_end(self):
        # 전체 에포크 데이터 수집
        all_preds = torch.cat(self.train_preds)
        all_targets = torch.cat(self.train_targets)
        
        # 에포크 평균 손실 계산
        epoch_loss = torch.stack(self.train_losses).mean()  # 중요!
        
        # 메트릭 계산
        epoch_acc = accuracy_score(all_targets.numpy(), all_preds.numpy())
        epoch_f1 = f1_score(all_targets.numpy(), all_preds.numpy(), average='macro')
        
        # 로깅 (epoch_loss 포함)
        self.log('train_loss', epoch_loss, prog_bar=True)
        self.log('train_acc', epoch_acc, prog_bar=True)
        self.log('train_f1', epoch_f1, prog_bar=True)
        
        # 다음 에포크를 위해 리셋
        self.train_preds.clear()
        self.train_targets.clear()
        self.train_losses.clear()  # 손실 리스트 초기화

    def predict_step(self, batch, batch_idx):
        x, _ = batch      
        y_hat = self(x)
        return y_hat.argmax(dim=1)

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.lr)
    #     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    #     optimizer, mode='min', factor=0.1, patience=5
    #       )
    #        return {
    #           "optimizer": optimizer,
    #           "lr_scheduler": {
    #               "scheduler": scheduler,
    #               "monitor": "val_loss"  # 검증 손실 기반
    #           }
    #       }
        return optimizer

lightning_model = LightningModel(CFS['MODEL'], num_classes, CFS["LR"])

# Train

In [59]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 2            |        cudaMalloc retries: 3         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  20858 MiB |  23378 MiB | 387905 GiB | 387885 GiB |
|       from large pool |  20844 MiB |  23351 MiB | 385347 GiB | 385327 GiB |
|       from small pool |     13 MiB |     98 MiB |   2557 GiB |   2557 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  20858 MiB |  23378 MiB | 387905 GiB | 387885 GiB |
|       from large pool |  20844 MiB |  23351 MiB | 385347 GiB |

In [60]:
# 트레이너 설정
trainer = Trainer(
    max_epochs=CFS["EPOCHS"],
    accelerator='cuda' if torch.cuda.is_available() else 'cpu',
    devices="auto",
    logger=wandb_logger,
    # callbacks=[
    #     pl.callbacks.ModelCheckpoint(
    #         dirpath="checkpoints/",
    #         filename="{epoch}-{val_loss:.2f}",
    #         save_top_k=3,
    #         monitor="val_loss"  # 검증 데이터 있을 때만 유효
    #     )
    # ]
)

# 학습 실행
trainer.fit(
    model=lightning_model,
    datamodule=datamodule
)

wandb.finish()


Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | ConvNeXt | 657 M  | train
-------------------------------------------
657 M     Trainable params
0         Non-trainable params
657 M     Total params
2,630.082 Total estimated model params size (MB)
465       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 23.75 MiB is free. Process 2196551 has 23.65 GiB memory in use. Of the allocated memory 23.24 GiB is allocated by PyTorch, and 98.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Prediction

In [33]:
predictions = trainer.predict(
    model=lightning_model, 
    datamodule=datamodule
)

# 4. 결과 처리
all_preds = torch.cat(predictions).cpu().numpy()  # [n_samples]
# 샘플 제출 파일 로드
submission = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
# 예측값으로 타겟 열 업데이트
submission["target"] = all_preds


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/data/ephemeral/home/.pyenv/versions/3.12.10/envs/py12/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=47` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [34]:

# 결과 저장
submission.to_csv("submission.csv", index=False)

# 모델저장, 불러오기 - 작업중

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

# 체크포인트 콜백 설정
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",               # 저장 경로
    filename="{epoch}-{val_loss:.2f}",    # 파일명 형식
    save_top_k=2,                         # 상위 2개 모델 저장
    monitor="train_loss",                   # 모니터링 지표
    mode="min",                           # 최소화 대상
    save_weights_only=False               # 전체 상태 저장 (권장)
)

# 트레이너에 콜백 추가
trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=CFS['EPOCHS'],
    logger=wandb_logger
)

# 특정 지점에서 수동 저장
trainer.save_checkpoint("manual_save.ckpt")

# 모델 아키텍처 초기화
loaded_model = LightningModel.load_from_checkpoint(
    checkpoint_path="checkpoints/epoch=10-val_loss=0.32.ckpt",
    model_name=CFS['MODEL'],
    num_classes=num_classes,
    lr=CFS['LR']
)

# 예측 실행
predictions = trainer.predict(model=loaded_model, datamodule=datamodule)

# 이전 체크포인트에서 학습 재개
trainer = Trainer(resume_from_checkpoint="checkpoints/last.ckpt")
trainer.fit(loaded_model, datamodule=datamodule)
